In [37]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import seaborn as sns

# Utiilities
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

# Constant
BASE_PATH = "DS/optiver-realized-volatility-prediction/"
TEST_FILE = BASE_PATH + "test.csv"
TRAIN_FILE = BASE_PATH + "train.csv"

N_FOLDS = 10
N_REPEAT = 3

In [2]:
# Financial utilities functions

def wap(df, index):
    """
    Compute the Waighted Avergae Price
    """
    bid_price = df[f"bid_price{index}"]
    bid_size = df[f"bid_size{index}"]
    ask_price = df[f"ask_price{index}"]
    ask_size = df[f"ask_size{index}"]
    return bid_price * ask_size + ask_price * bid_size / (bid_size + ask_size)

def log_return(series):
    """
    Compute the log_return of WAP
    """
    return np.log(series).diff()

def realized_volatility(series):
    """
    Compute the realized volatility
    """
    return np.sqrt(np.sum(series**2))

def total_volume(df):
    """
    Compute the total volume between asks and bids
    """
    ask_size_1 = df["ask_size1"]
    ask_size_2 = df["ask_size2"]
    bid_size_1 = df["bid_size1"]
    bid_size_2 = df["bid_size2"]
    return ask_size_1 + ask_size_2 + bid_size_1 + bid_size_2

def imbalance_volume(df):
    """
    Computes the imbalance volume
    """
    ask_size_1 = df["ask_size1"]
    ask_size_2 = df["ask_size2"]
    bid_size_1 = df["bid_size1"]
    bid_size_2 = df["bid_size2"]
    return np.abs(ask_size_1 + ask_size_2 - bid_size_1 - bid_size_2)

def spread_size(df, type):
    """
    Compute the spread
    """
    size_1 = df[f"{type}_size1"]
    size_2 = df[f"{type}_size2"]
    return size_1 - size_2

def spread_price(df, type):
    """
    Compute the spread
    """
    price_1 = df[f"{type}_price1"]
    price_2 = df[f"{type}_price2"]
    return price_1 - price_2

def price_spread(df, index):
    """
    Compute the price spred
    """
    ask_price = df[f"ask_price{index}"]
    bid_price = df[f"bid_price{index}"]
    return (ask_price - bid_price) / (ask_price + bid_price)

In [8]:
def set_book_features(df, window_size=100):
    """
    Set book features // Windowing rows on columns
    """
    # First level aggregation
    df["wap_1"]                = wap(df, 1)
    df["wap_2"]                = wap(df, 2)
    df["log_return_1"]         = df.groupby("time_id")["wap_1"].apply(log_return)
    df["log_return_2"]         = df.groupby("time_id")["wap_2"].apply(log_return)
    df["bid_size_spread"]      = spread_size(df, "bid")
    df["bid_price_spread"]     = spread_price(df, "bid")
    df["ask_size_spread"]      = spread_size(df, "ask")
    df["ask_price_spread"]     = spread_price(df, "ask")
    df["price_spread_1"]       = price_spread(df, 1)
    df["price_spread_2"]       = price_spread(df, 2)
    df['wap_balance']          = np.abs(df['wap_1'] - df['wap_2'])
    df["bid_ask_price_spread"] = np.abs(df["ask_price_spread"] - df["bid_price_spread"])
    df["bid_ask_size_spread"]  = np.abs(df["ask_size_spread"] - df["bid_size_spread"])
    df["total_volume"]         = total_volume(df)
    df["imbalance_volume"]     = imbalance_volume(df)
    
    # Second level aggregation
    aggregation_dict = {
        "log_return_1": [realized_volatility, np.mean],
        "log_return_2": [realized_volatility, np.mean],
        "wap_1": [np.mean, np.std],
        "wap_2": [np.mean, np.std],
        "bid_price1": [np.mean, np.max],
        "bid_price2": [np.mean, np.max],
        "ask_price1": [np.mean, np.min],
        "ask_price2": [np.mean, np.min],
        "bid_size1": [np.mean, np.sum],
        "bid_size2": [np.mean, np.sum],
        "ask_size1": [np.mean, np.sum],
        "ask_size2": [np.mean, np.sum],
        "bid_size_spread": [np.mean, np.std],
        "bid_price_spread": [np.mean, np.std],
        "ask_size_spread": [np.mean, np.std],
        "ask_price_spread": [np.mean, np.std],
        "price_spread_1": [np.mean, np.std],
        "price_spread_2": [np.mean, np.std],
        "wap_balance": [np.mean, np.std],
        "bid_ask_price_spread": [np.mean, np.std],
        "bid_ask_size_spread": [np.mean, np.std],
        "total_volume": [np.mean, np.std],
        "imbalance_volume": [np.mean, np.std]
    }
    features_lst = []
    for second in range(0, 600, window_size):
        df_feature = df[df['seconds_in_bucket'] >= second].groupby("time_id").agg(aggregation_dict)
        df_feature.columns = ['_'.join(col) + f"_s{600 - second}" for col in df_feature.columns]
        features_lst.append(df_feature)
    return pd.concat(features_lst, axis=1)

In [4]:
def set_trade_features(df, window_size=100):
    """
    Set trade features // Windowing rows on columns
    """
    # First level aggregation
    df["log_return"]   = df.groupby("time_id")["price"].apply(log_return)
    df['amount']       = df['price'] * df['size']
    
    # Second level aggregation
    aggregation_dict = {
        "log_return": [realized_volatility, np.mean],
        "size": [np.mean, np.sum],
        "amount": [np.mean, np.sum],
        "order_count": [np.mean, np.sum],
    }
    features_lst = []
    for second in range(0, 600, window_size):
        df_feature = df[df['seconds_in_bucket'] >= second].groupby("time_id").agg(aggregation_dict)
        df_feature.columns = ['_'.join(col) + f"_s{600 - second}" for col in df_feature.columns]
        features_lst.append(df_feature)
    return pd.concat(features_lst, axis=1)

In [5]:
def preprocess(ids_list, mode, window_size=60):
    """
    Preprocess dataset
    """
    def create_stock_df(stock_id):
        book = pd.read_parquet(f"{BASE_PATH}book_{mode}.parquet/stock_id={stock_id}")
        trade = pd.read_parquet(f"{BASE_PATH}trade_{mode}.parquet/stock_id={stock_id}")

        book_features = set_book_features(book, window_size)
        trade_features = set_trade_features(trade, window_size)
        
        features = book_features.join(trade_features, how='outer')
        features = features.reset_index()
        features["stock_id"] = stock_id
        return features
      
    df_list = Parallel(n_jobs=-1, verbose=1)(
        delayed(create_stock_df)(_id) for _id in ids_list
    )
    return pd.concat(df_list, ignore_index = True)

In [6]:
# Get the train file
train = pd.read_csv(TRAIN_FILE)
train_ids = train.stock_id.unique()

In [9]:
# Preprocess data + FE
train_df = preprocess(train_ids, "train")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 10.5min finished


In [15]:
# Merge and select
df_train = train.merge(train_df, on=["stock_id", "time_id"], how="left")

In [21]:
# Forward filling the missing
df_train = df_train.ffill()

In [35]:
X = df_train.drop("target", axis=1)
y = df_train["target"]

In [38]:
# Scaling

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = np.asarray(y)

In [25]:
# Model library
models = [
    LinearRegression(),
]

In [40]:
# get OOF prediction
def get_out_of_fold_predictions(X, y, models):
    meta_X, meta_y = list(), list()
    kfold = KFold(n_splits=N_FOLDS, shuffle=True)

    for train_ix, test_ix in kfold.split(X):
        fold_yhats = list()
        
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        meta_y.extend(test_y)

        for model in models:
            model.fit(train_X, train_y)
            yhat = model.predict(test_X)
            fold_yhats.append(yhat.reshape(len(yhat),1))

        meta_X.append(np.hstack(fold_yhats))
    return np.vstack(meta_X), np.asarray(meta_y)

In [27]:
def fit_base_models(X, y, models):
    for model in models:
        model.fit(X, y)

In [28]:
def fit_meta_model(X, y):
    model = LinearRegression()
    model.fit(X, y)
    return model

In [30]:
# Compute the Root Mean Squate Percentage Error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [32]:
def evaluate_models(X, y, models):
    for model in models:
        yhat = model.predict(X)
        _rmspe = rmspe(y, yhat)
        print('%s: RMSPE %.5f' % (model.__class__.__name__, rmspe))

In [33]:
def super_learner_predictions(X, models, meta_model):
    meta_X = list()
    for model in models:
        yhat = model.predict(X)
        meta_X.append(yhat.reshape(len(yhat),1))
    meta_X = np.hstack(meta_X)

    return meta_model.predict(meta_X)

In [ ]:
# Super"Learner" Call
fe, fe_test, label, label_test = train_test_split(X, y, test_size=.2)

meta_X, meta_y = get_out_of_fold_predictions(fe, label, models)
fit_base_models(fe, label, models)
meta_model = fit_meta_model(meta_X, meta_y)
evaluate_models(fe_test, label_test, models)
yhat = super_learner_predictions(fe_test, models, meta_model)
print('Super Learner: RMSPE %.5f' % rmspe(label_test, yhat))